In [82]:
import psycopg2
from psycopg2 import connect
from dotenv import load_dotenv
from os import environ
import pandas as pd
import textblob
from textblob import TextBlob
import plotly.graph_objects as go
import numpy as np
import streamlit
import json
from requests import get

In [32]:
def get_db_connection():
    """Returns a database connection."""
    load_dotenv()
    return psycopg2.connect(
        host=environ["DB_HOST"],
        port=environ["DB_PORT"],
        database=environ["DB_NAME"],
        user=environ["DB_USER"],
        password=environ["DB_PASSWORD"]
        )


In [67]:
# load the comment data from the most recent set of records

# Index(['record_id', 'story_id', 'score', 'comments', 'record_time', 'story_id',
#        'title', 'author', 'story_url', 'creation_date', 'topic_id'],
#       dtype='object')

def load_stories_data() -> pd.DataFrame:
    """Loads stories with greatest score change over last 24hrs from RDS.
    Returns them as a Dataframe object."""
    query = """
        SELECT stories.story_id, 
                stories.title, 
                stories.author,
                stories.story_url,
                records.comments, 
                records.record_id, 
                stories.topic_id,
                records.score
        FROM records
        LEFT JOIN stories ON records.story_id = stories.story_id
        WHERE record_time >= NOW() - INTERVAL '1 hours'
        ;
        """
    return pd.read_sql(query, con=get_db_connection())

In [73]:
load_dotenv()
df = load_stories_data()


/var/folders/2f/000yp3dd3t359b791hdtl7jm0000gp/T/ipykernel_79900/3940078497.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,story_id,title,author,story_url,comments,record_id,topic_id,score


## Retrieve comment string


In [95]:
BASE_URL = "https://hacker-news.firebaseio.com/v0/item/"


# def clean_API_text(comment: str) -> str:
#     # Replace HTML entities and convert the string to HTML
#     input_string = "\"Nice! Playing around with the widget under the section &quot;Experimenting with lines&quot; I&#x27;m reminded of Bret Victor&#x27;s Inventing on Principle talk [0] (an absolute must watch, if anyone hasn&#x27;t yet). In particular, changing the smoothness reveals a sort of scaling effect that I&#x27;d probably never know about if not playing around with sliders and having it update in real time rather than setting individual values. Very interesting and beautiful!<p>[0] <a href=\"https:&#x2F;&#x2F;youtu.be&#x2F;PUv66718DII?si=2urxGUwD_lWA8C4q\" rel=\"nofollow\">https:&#x2F;&#x2F;youtu.be&#x2F;PUv66718DII?si=2urxGUwD_lWA8C4q</a>\""


def get_comment_ids(story: int) -> list:
    """Returns the id of comments from given story on Hacker News."""
    story_info = get(BASE_URL + f"{story}.json", timeout=30).json()
    comment_ids = story_info.get("kids")
    return comment_ids


def get_comments(comment: int) -> str:
    """Returns the text from a given comment."""

    comment_info = get(BASE_URL + f"{comment}.json", timeout=30).json()
    comment = comment_info.get("text")
    return comment

In [97]:
comments = get_comment_ids(38865518)
comment_list = []
for comment in comments:
    comment_list.append(get_comments(comment))



### Sentiment analysis for each comment

In [114]:

sentiment_list = []
for comment in comment_list:
    blob = TextBlob(str(comment))
    blob.tags           
    blob.noun_phrases   
    for sentence in blob.sentences:
        sentiment = sentence.sentiment.polarity
    sentiment_list.append({'comment': comment,
    'sentiment': sentiment})
sentiment_mean = []
for i in range(0,20):
    sentiment_mean.append(sentiment_list[i]['sentiment'])
average = sum(sentiment_mean)/len(sentiment_mean)
average


-0.05570238095238096

In [125]:

value = average

if -1 <= value < -0.5:
    color="#FF0000"
elif -0.49 <= value < 0:
    color="#FFA500"
elif 0.1 <= value < 1:
    color="#FFA500"

fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=value,
    title={'text': "Overall Sentiment"},
    gauge={'axis': {'range': [-1, 1]},
        'bar': {'color': f'{color}'},
        }
))
fig.update_layout(height=350) 

# st.plotly_chart(fig, use_container_width=True)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'gauge': {'axis': {'range': [-1, 1]}, 'bar': {'color': '#FFA500'}},
              'mode': 'gauge+number',
              'title': {'text': 'Overall Sentiment'},
              'type': 'indicator',
              'value': -0.05570238095238096}],
    'layout': {'height': 350, 'template': '...'}
})

In [17]:
text = '''
I am very very happy.
'''

blob = TextBlob(text)
blob.tags           

blob.noun_phrases   
for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

1.0
